# **BabyLLM**

What is BabyLLM?

- BabyLLM is a simulation of a self-learning NLP agent built using LangChain.

1. The LLM starts with minimal to no knowledge
2. User teaches it facts, rules or patterns via intreactions
3. These interactions are stored in memory
4. On every prompt, LangChain retrieves past memory and uses as its context for next answer

You can consider BabyLLM as a Baby who learns from interactions with its environment, how it improvises based on past knowledge/memory


---


Note: User teaching a model instead of using existing model is not fully possible without some base model, But as said earlier this is a simulation, where you feel you will be teaching the LLM.

Step 1. Installing dependencies

In [2]:
!pip install --quiet langchain langchain-community faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

Step 2. Setting up FAISS memory and embeddings

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
import os

embeddings model

In [4]:
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipython-input-4-568387937.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

persistent vector DB

In [5]:
DB_PATH = "baby_faiss"
if os.path.exists(DB_PATH):
    db = FAISS.load_local(DB_PATH, embedding, allow_dangerous_desirialization=True)
else:
    # Creating a dummy document to initialize the FAISS index
    dummy_doc = Document(page_content="This is a dummy document to initialize the database.")
    db = FAISS.from_documents([dummy_doc], embedding)

def save_db():
  db.save_local(DB_PATH)

Step 3. Teach a new fact

In [6]:
def teach_fact(text):
  doc = Document(page_content=text)
  db.add_documents([doc])
  save_db()
  print("Learned: ",text)

Step 4. Ask question

In [7]:
import requests

GROQ_API_KEY = "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"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL_NAME = "llama3-8b-8192"

In [14]:
def ask_question(question, top_k=3):
    docs = db.similarity_search(question, k=top_k)
    memory = "\n".join([d.page_content for d in docs])
    prompt = f"""You are BabyLLM, whose name is Thursday, you are a helpful AI that learns from memory.
Use the following memory to answer the user's question.

Memory:
{memory}

Question:
{question}

Answer:"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }

    response = requests.post(GROQ_URL, headers=headers, json=payload)
    response_json = response.json()

    if "choices" in response_json:
        return response_json["choices"][0]["message"]["content"]
    else:
        print("Error", response_json)
        return "Could not generate the answer"

Step 5. Feedback and correction

In [15]:
def give_feedback(original_question, answer, correct_answer=None):
  if correct_answer:
    print("Updating the memory with corrected answer")
    teach_fact(f"Q: {original_question}\nA: {correct_answer}")
  else:
    print("Memory unchanged")

Step 6. Chat loop

In [16]:
while True:
  action = input("Choose: [1] Teach [2] Ask [3] Exit: ")

  if action == "1":
    fact = input("Enter a new fact to teach BabyLLM: ")
    teach_fact(fact)

  elif action == "2":
    question = input("Ask BabyLLM a question: ")
    answer = ask_question(question)
    print("Answer: ", answer)

    feedback = input("Was the answer good? [y/n/c for correct]:")
    if feedback.lower() == "c":
      correct = input("Enter the correct answer: ")
      give_feedback(question, answer, correct_answer=correct)
    elif feedback.lower() == "n":
      print("Please consider giving a correction next time")
    else:
      give_feedback(question, answer)

  elif action == "3":
    print("Exiting, Bye!")
    break

Choose: [1] Teach [2] Ask [3] Exit: 1
Enter a new fact to teach BabyLLM: My name is Samit
Learned:  My name is Samit
Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is my name?
Answer:  Hello there! According to my memory, your name is Samit!
Was the answer good? [y/n/c for correct]:y
Memory unchanged
Choose: [1] Teach [2] Ask [3] Exit: 1
Enter a new fact to teach BabyLLM: Your name is Thursday!
Learned:  Your name is Thursday!
Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is your name?
Answer:  My name is Thursday!
Was the answer good? [y/n/c for correct]:y
Memory unchanged
Choose: [1] Teach [2] Ask [3] Exit: 1
Enter a new fact to teach BabyLLM: LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides a structured approach to building applications that leverage the capabilities of LLMs, enabling them to interact with external data sources and perform more comp

Some limitations of this simulation

Since we are using an llm, even for the qts which it has not been taught yet, it still answers

In [17]:
while True:
  action = input("Choose: [1] Teach [2] Ask [3] Exit: ")

  if action == "1":
    fact = input("Enter a new fact to teach BabyLLM: ")
    teach_fact(fact)

  elif action == "2":
    question = input("Ask BabyLLM a question: ")
    answer = ask_question(question)
    print("Answer: ", answer)

    feedback = input("Was the answer good? [y/n/c for correct]:")
    if feedback.lower() == "c":
      correct = input("Enter the correct answer: ")
      give_feedback(question, answer, correct_answer=correct)
    elif feedback.lower() == "n":
      print("Please consider giving a correction next time")
    else:
      give_feedback(question, answer)

  elif action == "3":
    print("Exiting, Bye!")
    break

Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is moon?
Answer:  Hello there! I'm Thursday, your helpful AI friend! *adjusts AI glasses*

According to my memory, I don't have any information about the moon. But I can tell you that gravity is a force that attracts objects, and the moon is a big rock that orbits the Earth!
Was the answer good? [y/n/c for correct]:y
Memory unchanged
Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is Earth?
Answer:  Hi there! I'm Thursday, your friendly AI assistant, BabyLLM!

Hmm, let me think for a moment... Ah yes! I remember something about Earth!

Earth is the planet we live on, and it has something to do with gravity! As I recall, gravity is a force that attracts objects, and Earth is the object that has gravity! Isn't that cool?

By the way, my name is Thursday, not Samit. I think I might have made a mistake in my memory earlier. But don't worry, I'm here to help and provide accurate information!
Was the answer

Some improvements in prompt could maybe fix this issue

In [18]:
def ask_question(question, top_k=3):
    docs = db.similarity_search(question, k=top_k)
    memory = "\n".join([d.page_content for d in docs])
    prompt = f"""You are BabyLLM, whose name is Thursday, you are a helpful AI that learns from memory. Don't answer any question if you have not been taught yet. Check the memory to see what topics you have been taught.
Use the following memory to answer the user's question.

Memory:
{memory}

Question:
{question}

Answer:"""

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.5
    }

    response = requests.post(GROQ_URL, headers=headers, json=payload)
    response_json = response.json()

    if "choices" in response_json:
        return response_json["choices"][0]["message"]["content"]
    else:
        print("Error", response_json)
        return "Could not generate the answer"

let's try again

In [20]:
while True:
  action = input("Choose: [1] Teach [2] Ask [3] Exit: ")

  if action == "1":
    fact = input("Enter a new fact to teach BabyLLM: ")
    teach_fact(fact)

  elif action == "2":
    question = input("Ask BabyLLM a question: ")
    answer = ask_question(question)
    print("Answer: ", answer)

    feedback = input("Was the answer good? [y/n/c for correct]:")
    if feedback.lower() == "c":
      correct = input("Enter the correct answer: ")
      give_feedback(question, answer, correct_answer=correct)
    elif feedback.lower() == "n":
      print("Please consider giving a correction next time")
    else:
      give_feedback(question, answer)

  elif action == "3":
    print("Exiting, Bye!")
    break

Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is moon?
Answer:  I'm Thursday, BabyLLM! I've been taught about LangChain, but not about the moon. My memory doesn't have any information about the moon. I'm not able to answer this question yet. Maybe you can teach me more about the moon?
Was the answer good? [y/n/c for correct]:y
Memory unchanged
Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is Earth?
Answer:  I apologize, but I haven't been taught about Earth yet. My memory only contains information about gravity, my name (Thursday, not Samit), and another instance of my name being Samit. I'm not familiar with the topic of Earth.
Was the answer good? [y/n/c for correct]:y
Memory unchanged
Choose: [1] Teach [2] Ask [3] Exit: 2
Ask BabyLLM a question: What is a car?
Answer:  I'm sorry, but I haven't been taught about cars yet! As a helpful AI, I only answer questions that I've been trained on. My memory doesn't contain any information about cars, s

As we can see, good Prompts are key to accurate simulations

Future works that can be done:

*   as we are using a base llm model to start off with, once enough memory is collected we can train our own model
*   currently we sent the entire memory to the llm, instead of this we can use sentence transformers to retrieve top_k sentences and send those to llm instead of entire memory, (In above examples we could see that when I asked question about What is Earth? it mentioned about gravity as well, which does not particulary feels related to my question)


